## Flujo de Trabajo en NLP

<img src="./_src/PLN_Flujo_Trabajo.jpg" height="300"><br>

In [39]:
import pandas as pd
import nltk
import itertools
import re 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer



In [40]:
# Asegurarse de haber descargado los recursos necesarios
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

In [41]:
# Cargo la información de las reviews
df_reviews = pd.read_parquet('.\\Dataset\\user_reviews.parquet')
df_reviews = df_reviews.iloc[:1000]

In [42]:
df_reviews.shape

(1000, 8)

In [43]:
df_reviews['recommend'].value_counts()

recommend
True     875
False    123
Name: count, dtype: int64

# Cargo las Stopwords en Inglés

In [44]:
stopwords = nltk.corpus.stopwords.words('english') 

In [45]:
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

# Función para mapeo de tipo de palabra para Lematización

In [46]:
# Las etiquetas refieren al tipo de palabra. Vamos a definir una función para traducir estas etiquetas a los valores de POS que entiende 'wordnet_lemmatizer'.
def get_wordnet_pos(word):
    
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    print(tag)     
    return tag_dict.get(tag, wordnet.NOUN)

# TOKENIZACIÓN, EXPRESIONES REGULARES y LEMATIZCIÓN

In [47]:
# Las etiquetas refieren al tipo de palabra. Vamos a definir una función para traducir estas etiquetas a los valores de POS que entiende 'wordnet_lemmatizer'.
def get_wordnet_pos(word):
    
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)



def process_review(review):
    if pd.isna(review):
        return []  # Review vacío
    
    # Limpieza y procesamiento
    # Si la review no es nula
    
    # Eliminar todo lo que no sean letras y convertir a minúsculas
    review = re.sub("[^a-zA-Z]", " ", str(review)).lower()
    # Tokeniza la review depues de sacar las condiciones regulares que le pasamos
    tokenizer = RegexpTokenizer(r'\w+')
    review = tokenizer.tokenize(review)
    # Elimina las stopwords 
    review = [word for word in review if word not in stopwords]
    # Lematización
    review = [wordnet_lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in review]
    
    return review


# Aplicar la función de procesamiento a cada reseña
df_reviews["processed_reviews"] = df_reviews["review"].apply(process_review)


In [56]:
from transformers import pipeline
from transformers import AutoTokenizer
import pandas as pd

# Cargar el modelo de análisis de sentimientos
#sentiment_analysis = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
sentiment_analysis = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", batch_size=16)

# Debido a que hay reviews con más de 512 tokens, debo importar AutoTkenizer para limitar la cantidad.
# Cargar el tokenizador asociado con el modelo
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [57]:
# Aplicar el análisis de sentimiento a las reviews procesadas
sentiment_results = []

# Debido a que el pipeline "sentiment-analysis" clasifica en POSITIVE o NEGATIVE, se define la siguiente función:
# Función para clasificar una reseña como positiva, neutra o negativa
def classify_review(review):
    # Unir la lista de palabras en una cadena de texto
    review_text = " ".join(review)

    if not review_text:
        return 'Neutro' # Si no hay review, asignar "Neutro"
    
    # Tokenizar la review y truncar si es necesario porque transformer requiere máx 512
    inputs = tokenizer(review_text, truncation=True, max_length=512, return_tensors="pt")
    
    # Realizar el análisis de sentimiento
    result = sentiment_analysis(tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True))[0]
    label = result['label']
    score = result['score']

    # Convertir las etiquetas a Positivo, Neutro y Negativo
    if label == 'POSITIVE' and score > 0.75:
        return 'Positivo'
    elif label == 'POSITIVE' and score <= 0.75:
        return 'Neutro'
    elif label == 'NEGATIVE' and score > 0.75:
        return 'Negativo'
    else:
        return 'Neutro'



# Aplicar el análisis de sentimiento a las reviews procesadas
df_reviews['sentiment'] = df_reviews['processed_reviews'].apply(lambda review: classify_review(review) if review else 'Neutro')

In [52]:
df_reviews['sentiment']

0      Positivo
1      Positivo
2      Positivo
3      Positivo
4      Positivo
         ...   
995    Negativo
996    Positivo
997    Positivo
998      Neutro
999    Positivo
Name: sentiment, Length: 1000, dtype: object